# 1. 개발 환경 설정

### 1.1 필수 라이브러리 설치하기

In [1]:
!pip3 install -q -U transformers
!pip3 install -q -U datasets
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.5 M

In [2]:
# prompt: i want to print out the versions only

!nvidia-smi
!echo "bitsandbytes version:"
!pip show bitsandbytes
!echo "peft version:"
!pip show peft
!echo "trl version:"
!pip show trl
!echo "accelerate version:"
!pip show accelerate
!echo "datasets version:"
!pip show datasets
!echo "transformers version:"
!pip show transformers

Wed Aug 14 10:25:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
# 구글 드라이브 마운트

from google.colab import drive
import shutil
import os

drive.mount('/content/drive')

Mounted at /content/drive


### 1.2 Import modules

In [4]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [5]:
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm import tqdm
from datasets import DatasetDict
from accelerate import Accelerator

# 2. Dataset 생성 및 준비

### 2.0 csv파일 불러오기

In [6]:
# open.zip 파일을 업로드한 경로를 확인하여 적용
!unzip  "/content/drive/MyDrive/2024인하대/open.zip"

Archive:  /content/drive/MyDrive/2024인하대/open.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [7]:
file_path = '/content/train.csv'
train_data = pd.read_csv(file_path)

train_data = train_data.sample(frac=1).reset_index(drop=True)

train_data.head(2)

,id,context,question,answer
0,TRAIN_26076,충남 보령시는 2021년 1월 1일 기준 개별공시지가에 대해 오는 26일까지 열람 ...,충남 보령시에서 개별공시지가에 대한 감정평가사가 검사하여 증명을 마친 땅은 얼마나 돼,24만 여 필지
1,TRAIN_17504,연초부터 재계 총수 신년사에서 빠지지 않고 등장한 키워드는 ‘ESG’다. 일반인에겐...,ESG 실천에 제일 능동적인 모습을 보이는 기업은 어디야,SK


In [8]:
train_data = train_data[train_data['context'].apply(len) <= 1300]

In [9]:
train_data

,id,context,question,answer
0,TRAIN_26076,충남 보령시는 2021년 1월 1일 기준 개별공시지가에 대해 오는 26일까지 열람 ...,충남 보령시에서 개별공시지가에 대한 감정평가사가 검사하여 증명을 마친 땅은 얼마나 돼,24만 여 필지
2,TRAIN_24944,지역 대규모의 울산웨딩박람회(주관 울산결만사 대표 조봉제)가 오는 3월13~14일 ...,3월에 울산에서 2일 동안 열리는 대형 행사는 뭐지,울산웨딩박람회
3,TRAIN_33362,"혼다의 프리미엄 세단, 어큐라 TLX가 더욱 강렬한 매력을 부여 받았다.\n이번에 ...",스포티하게 변신한 혼다 제품이 뭐야,어큐라 TLX 타입 S
4,TRAIN_04639,강원도는 도내 우수 제품 홍보·판매 지원을 위한 '2021 소셜마케팅 카페트' 2호...,두 번째로 강원도의 소셜마케팅 지원을 받게 된 제품은 어떤 기업이 만들어,혜성식품
6,TRAIN_05180,갤러리아백화점 광교점은 올해 첫 ‘봄 정기세일’을 다음 달 2일부터 18일까지 진행...,갤러리아 백화점은 경제적 어려움을 겪고 있는 농가를 돕기 위해 어떤 행사를 주최해,갤러리아와 함께하는 농가동행 봄
...,...,...,...,...
33711,TRAIN_22547,정치권의 공매도 금지 연장 압박이 계속되는 가운데 금융당국이 개인 투자자들의 공매도...,유월에 끝날 것으로 예상되는 금융당국의 공매도 관련 체제는 어떤 거야,통합 개인 대주 시스템
33712,TRAIN_20738,전라북도경제통상진흥원(원장 조지훈)이 지속성장 가능한 사회적경제기업을 육성하고 창업...,어떤 기관에서 사회적기업의 창업 과정을 지원하고 있어,경진원
33713,TRAIN_06759,구미시가 수출기업의 경영안정화 및 해외시장 개척을 위해 중소기업 수출보험료 지원사업...,경상북도에서 중소규모의 해외 판매 회사는 관련 보험료를 도움받을 수 있는 금액은 얼마야,최대 600만 원
33714,TRAIN_04734,"강원도의 수출 4대 효자품목(의료기기, 자동차부품, 시멘트, 합금철)이 코로나19로...",강원도의 수출 4대 효자품목은 무엇인가,"의료기기, 자동차부품, 시멘트, 합금철"


In [10]:
dataset = Dataset.from_pandas(train_data)
dataset

Dataset({
    features: ['id', 'context', 'question', 'answer', '__index_level_0__'],
    num_rows: 28574
})

In [11]:
dataset[0]

{'id': 'TRAIN_26076',
 'context': '충남 보령시는 2021년 1월 1일 기준 개별공시지가에 대해 오는 26일까지 열람 및 의견을 접수한다고 밝혔다. \n\n시는 올해 1월부터 토지특성조사 및 산정 절차를 거쳐 최종적으로 24만 여 필지의 개별공시지가에 대한 감정평가사의 검증이 끝남에 따라 결정ㆍ공시 전에 열람 및 의견 신청을 받는다. \n\n개별공시지가는 부동산공시가격알리미(https://www.realtyprice.kr)에서 확인할 수 있으며, 의견이 있는 토지소유자나 이해관계인은 오는 26일까지 시 민원지적과(☏ 930-3476, 팩스 930-3499) 또는 읍ㆍ면ㆍ동에 의견제출서를 작성해 제출하면 된다. \n\n의견이 제출된 필지는 인근 토지, 표준지와의 가격 균형 여부 등을 재조사하고 감정평가사 검증과 보령시 부동산가격공시위원회의 심의를 거쳐 다음달 10일까지 개별 통지한다. \n\n보령시 관계자는 "개별공시지가는 토지와 관련된 각종 과세 및 부담금의 기준으로 활용되므로 많은 시민이 열람하고 의견을 제출해 주길 바란다"고 당부했다.',
 'question': '충남 보령시에서 개별공시지가에 대한 감정평가사가 검사하여 증명을 마친 땅은 얼마나 돼',
 'answer': '24만 여 필지',
 '__index_level_0__': 0}

# 3. eeve 파인튜닝

#### 주의: Colab GPU 메모리 한계로 이전장 추론에서 사용했던 메모리를 비워 줘야 파인튜닝을 진행 할 수 있습니다. <br> notebook 런타임 세션을 재시작 한 후 1번과 2번의 2.1 항목까지 다시 실행하여 로드 한 후 아래 과정을 진행합니다

In [12]:
# !nvidia-smi
import subprocess

# GPU 상태 확인
def check_gpu():
    try:
        result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
        print(result.stdout.decode('utf-8'))
    except FileNotFoundError:
        print("nvidia-smi not found, ensure you have a GPU instance.")

check_gpu()

Wed Aug 14 10:26:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

### 3.1 학습용 프롬프트 조정

In [13]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['context'])):
        prompt_list.append(r"""<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해.:

{}

Question:
{}<|im_end|>
<|im_start|>assistant
{}<|im_end|><eos>""".format(example['context'][i], example['question'][i], example['answer'][i]))
    return prompt_list

In [14]:
train_data = dataset
print(generate_prompt(train_data[:1])[0])

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해.:

충남 보령시는 2021년 1월 1일 기준 개별공시지가에 대해 오는 26일까지 열람 및 의견을 접수한다고 밝혔다. 

시는 올해 1월부터 토지특성조사 및 산정 절차를 거쳐 최종적으로 24만 여 필지의 개별공시지가에 대한 감정평가사의 검증이 끝남에 따라 결정ㆍ공시 전에 열람 및 의견 신청을 받는다. 

개별공시지가는 부동산공시가격알리미(https://www.realtyprice.kr)에서 확인할 수 있으며, 의견이 있는 토지소유자나 이해관계인은 오는 26일까지 시 민원지적과(☏ 930-3476, 팩스 930-3499) 또는 읍ㆍ면ㆍ동에 의견제출서를 작성해 제출하면 된다. 

의견이 제출된 필지는 인근 토지, 표준지와의 가격 균형 여부 등을 재조사하고 감정평가사 검증과 보령시 부동산가격공시위원회의 심의를 거쳐 다음달 10일까지 개별 통지한다. 

보령시 관계자는 "개별공시지가는 토지와 관련된 각종 과세 및 부담금의 기준으로 활용되므로 많은 시민이 열람하고 의견을 제출해 주길 바란다"고 당부했다.

Question:
충남 보령시에서 개별공시지가에 대한 감정평가사가 검사하여 증명을 마친 땅은 얼마나 돼<|im_end|>
<|im_start|>assistant
24만 여 필지<|im_end|><eos>


### 3.2 QLoRA 설정

In [15]:
lora_config = LoraConfig(
    r=6,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [16]:
BASE_MODEL = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

### 3.3 Trainer 실행

In [17]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=None,
    args=TrainingArguments(
        output_dir="eeve_results",
        # 빠른 테스트를 위해 epoch 값 1로 고정
        num_train_epochs = 1,
        max_steps=-1,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        warmup_ratio=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1000,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/28574 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss


### 3.4 Finetuned Model 저장

In [ ]:
ADAPTER_MODEL = "/content/drive/MyDrive/eeve_lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)